In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
SAVE_TIME=False

In [3]:
x_train_list = pickle.load(open('../gen_data/x_train--features2.ipynb--.pickle','rb'))
x_test_list = pickle.load(open('../gen_data/x_test--features2.ipynb--.pickle','rb'))

In [4]:
sales = pd.read_csv('../original_data/sales_train.csv.gz')

# Generate time specific market price data

In [5]:
if not SAVE_TIME:
    months = range(sales.date_block_num.min() , sales.date_block_num.max() +1 )
    #months=range(1,2)
    #months=range(19,20)
    to_df=[]
    for month in months:
        print(month)
        dfm = sales[sales.date_block_num==(month-1)]
        items= dfm.item_id.unique()
        prices=[]
        for item in items:
            dfmi = dfm[dfm.item_id==item]
            pricesum = dfmi.item_price.sum()
            count = dfmi.item_cnt_day.sum()

            #sdprice=np.nan

            if count>0:
                meanprice=pricesum/count
            else:
                meanprice=np.nan
                
            if count>1:
                ss = np.sum(dfmi.item_cnt_day*(dfmi.item_price-meanprice)**2)
                sdprice = np.sqrt( ss/(count-1) )
            else:
                sdprice=np.nan
            #sdprice=np.sqrt(dfmi.item_cnt_day.sum()*(price - meanprice)**2/(count-1))
            to_df.append([month,item,meanprice,sdprice])

    price_data = pd.DataFrame(np.vstack(to_df),columns=['date_block_num','item_id','mean_prevmonth_item_price','sd_prevmonth_item_price'])

0
1
2
3
4
5


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in sqrt


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [6]:
#save time
if SAVE_TIME:
    price_data = pd.read_csv('../gen_data/price_data_for_features3.csv')
else:
    price_data.to_csv('../gen_data/price_data_for_features3.csv',index=False)

In [7]:
price_data['date_block_num'] = price_data.date_block_num.astype(np.int32)
price_data['item_id'] = price_data.item_id.astype(np.int32)
price_data.head()

,date_block_num,item_id,mean_prevmonth_item_price,sd_prevmonth_item_price
0,1,22154,999.00,0.000000
1,1,2552,NaN,NaN
2,1,2554,1709.05,NaN
3,1,2555,1098.85,0.212132
4,1,2564,327.60,47.851855


# Generate shop-time price data

In [8]:
#not gonna worry about getting mean price by transaction, will 
#settle for mean price by row (which could incnlude many transactions)
gp = sales.groupby(['date_block_num','shop_id','item_id'],as_index=False).item_price.mean()
gp['date_block_num'] += 1
gp.columns = ['date_block_num','shop_id','item_id','mean_prevmonth_shop_item_price']
gp.head()

,date_block_num,shop_id,item_id,mean_prevmonth_shop_item_price
0,1,0,32,221.0
1,1,0,33,347.0
2,1,0,35,247.0
3,1,0,43,221.0
4,1,0,51,128.5


In [9]:
market_data = price_data
shop_item_data = gp
def merge_market_data(df):
    return df.merge(market_data,how='left').merge(shop_item_data,how='left')

In [10]:
new_x_train_list = list(map(merge_market_data,x_train_list))
new_x_test_list = list(map(merge_market_data,x_test_list))

In [11]:
def insert_sd_above_market(df):
    df['sd_above_market'] = (df.mean_prevmonth_shop_item_price - df.mean_prevmonth_item_price) / df.sd_prevmonth_item_price
    return df

In [12]:
def cleanup(df):
    mean_sd = np.nanmean(df.sd_prevmonth_item_price)
    print(mean_sd)
    df['sd_prevmonth_item_price'] = df.sd_prevmonth_item_price.fillna(mean_sd)
    df['sd_above_market'] = df.sd_above_market.fillna(0)
    return df

In [13]:
new_x_train_list = list(map(insert_sd_above_market,new_x_train_list))
new_x_test_list = list(map(insert_sd_above_market,new_x_test_list))

In [14]:
new_x_train_list[1]

,date_block_num,shop_id,item_id,item_category_id,sum_item_sales_back_1,sum_shop_sales_back_1,item_cnt_month_back_1,sum_item_cat_sales_back_1,sum_item_cat_shop_sales_back_1,sum_item_sales_back_2,...,sum_shop_sales_back_12,item_cnt_month_back_12,sum_item_cat_sales_back_12,sum_item_cat_shop_sales_back_12,month,numdays,mean_prevmonth_item_price,sd_prevmonth_item_price,mean_prevmonth_shop_item_price,sd_above_market
0,12,2,32,40.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,jan,31,134.691573,15.602202,NaN,NaN
1,12,2,33,37.0,42.0,1255.0,1.0,9906.0,55.0,15.0,...,1146.0,1.0,5982.0,52.0,jan,31,186.571429,16.449056,199.000,0.755580
2,12,2,99,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,jan,31,124.167083,25.367019,NaN,NaN
3,12,2,482,73.0,88.0,1255.0,1.0,412.0,3.0,85.0,...,1146.0,1.0,664.0,7.0,jan,31,2866.022727,435.543311,3300.000,0.996404
4,12,2,485,73.0,111.0,1255.0,1.0,412.0,3.0,0.0,...,0.0,0.0,0.0,0.0,jan,31,250.203390,51.145216,300.000,0.973632
5,12,2,804,49.0,26.0,1255.0,1.0,842.0,7.0,0.0,...,1146.0,1.0,915.0,8.0,jan,31,227.115385,20.874165,240.000,0.617252
6,12,2,839,73.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1146.0,1.0,664.0,7.0,jan,31,3091.707317,344.029210,NaN,NaN
7,12,2,1007,67.0,247.0,1255.0,1.0,3576.0,10.0,0.0,...,1146.0,3.0,2202.0,28.0,jan,31,441.207977,155.515705,599.000,1.014637
8,12,2,1406,28.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,jan,31,758.350758,235.080954,NaN,NaN
9,12,2,1407,19.0,466.0,1255.0,13.0,14490.0,278.0,6.0,...,0.0,0.0,0.0,0.0,jan,31,734.125609,542.392120,1299.000,1.041450


In [15]:
new_x_train_list = list(map(cleanup,new_x_train_list))
new_x_test_list = list(map(cleanup,new_x_test_list))

60.19260224032919
61.17748807693945
95.06194872748173
nan


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


In [18]:
new_x_train_list[0].shape

(6186922, 70)

In [19]:
pickle.dump(new_x_train_list,open('../gen_data/x_train--features3.ipynb--.pickle','wb'))
pickle.dump(new_x_test_list,open('../gen_data/x_test--features3.ipynb--.pickle','wb'))

# End non experimental part

In [20]:
sales[(sales.date_block_num==1) & (sales.item_id==4124)].item_price.std()

35.32987404449668

In [21]:
sales[(sales.date_block_num==1) & (sales.item_id==4124)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
115693,13.02.2013,1,50,4124,249.0,1.0
115773,28.02.2013,1,50,4124,249.0,1.0
118044,24.02.2013,1,47,4124,249.0,1.0
118056,10.02.2013,1,47,4124,249.0,1.0
121688,02.02.2013,1,51,4124,170.0,1.0
122592,19.02.2013,1,42,4124,249.0,1.0
155866,06.02.2013,1,25,4124,249.0,1.0
166698,24.02.2013,1,3,4124,249.0,1.0
174790,19.02.2013,1,0,4124,170.0,1.0
174791,03.02.2013,1,0,4124,170.0,1.0


In [22]:
dfmi.item_price.sum()/dfmi.item_cnt_day.sum()

1799.0

In [23]:
dfmi

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882331,19.09.2015,32,25,21105,1799.0,1.0


In [24]:
pd.DataFrame(np.vstack(to_df),columns=['date_block_num','item_id','mean_cur_price','sd_cur_price']).head()

,date_block_num,item_id,mean_cur_price,sd_cur_price
0,1.0,22154.0,999.00,0.000000
1,1.0,2552.0,NaN,NaN
2,1.0,2554.0,1709.05,NaN
3,1.0,2555.0,1098.85,0.212132
4,1.0,2564.0,327.60,47.851855


In [25]:
sales[(sales.date_block_num==1) & (sales.item_id==3880)].item_price.sum()

64924.0

In [26]:
sales[(sales.date_block_num==1) & (sales.item_id==3880)].item_cnt_day.sum()

46.0

In [27]:
64924/46

1411.391304347826

In [28]:
#wont be exactly the same
sales[(sales.date_block_num==1) & (sales.item_id==3880)].item_price.mean()

1475.5454545454545

In [29]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [30]:
#sales = sales[['date_block_num','shop_id','item_id','item_price']]
sales['daily_rev'] = sales.item_price*sales.item_cnt_day
sales.shape


(2935849, 7)

In [31]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,daily_rev
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,-1.0,-899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00


In [32]:
#extend data longways such that item_cnt_day is always 1

In [33]:
#for row in range(0,sales.shape[0]):
#    print(row)
#    row_data = sales.loc[row]
#    icd = row_data.item_cnt_day
#    print(type(icd))
#    while icd > 1:
#        sales = pd.concat([sales,row_data])
#        icd -= 1

In [34]:
sales.groupby(['date_block_num','item_id'])

In [35]:
#months = range(sales.date_block_num.min() , sales.date_block_num.max() +1 )
#for month in months:
#    print(month)
#    mdf = sales[sales.date_block_num==month]
    

In [36]:
sales

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,daily_rev
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,-1.0,-899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
5,10.01.2013,0,25,2564,349.00,1.0,349.00
6,02.01.2013,0,25,2565,549.00,1.0,549.00
7,04.01.2013,0,25,2572,239.00,1.0,239.00
8,11.01.2013,0,25,2572,299.00,1.0,299.00
9,03.01.2013,0,25,2573,299.00,3.0,897.00
